In [ ]:
from scanbox.nlw_controller import *
from scanbox.utils import *
config = get_config()

box = ScanboxController('COM4',preferences = config['twophoton'])

[24-11-04 13:31:25] Could not connect to box on COM4
could not open port 'COM4': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)


OSError: Could not connect to Neurolabware Control Box

In [15]:
box.scan()

NameError: name 'box' is not defined

In [ ]:
box.abort()
box.set_mirror_position(0)

In [ ]:
box.exit_flag = True
box.usb.close()

In [ ]:
import atsapi as ats
board = ats.Board(systemId=1, boardId=1)
# Configure the capture clock (trigger external/laser)
board.setCaptureClock(source = ats.FAST_EXTERNAL_CLOCK,
                      rate = ats.SAMPLE_RATE_USER_DEF,
                      edge = ats.CLOCK_EDGE_RISING,
                      decimation = 0)                        

In [ ]:

channel_list = [ats.CHANNEL_A,ats.CHANNEL_B,ats.CHANNEL_C,ats.CHANNEL_D]
# configure the PMT channels
for channel in channel_list:
    board.inputControl(channel=channel,
                      coupling= ats.DC_COUPLING,
                      inputRange = ats.INPUT_RANGE_PM_1_V, 
                      impedance = ats.IMPEDANCE_50_OHM)
# Configure the trigger
board.setExternalClockLevel(65) # level in percent for the external clock (laser)
hsync_value = 0
trigger_level = 160
board.setTriggerOperation(operation = ats.TRIG_ENGINE_OP_J,
                          engine1 = ats.TRIG_ENGINE_J,
                          source1 = ats.TRIG_EXTERNAL,
                          slope1 = ats.TRIGGER_SLOPE_POSITIVE + int(hsync_value), # this depends on the HSYNC
                          level1 = int(trigger_level), # trigger level from 0 (-range) to 255 (+range)
                          engine2 = ats.TRIG_ENGINE_K,
                          source2 = ats.TRIG_DISABLE,
                          slope2 = ats.TRIGGER_SLOPE_POSITIVE,
                          level2 = 128)

board.setExternalTrigger(coupling = ats.DC_COUPLING,
                        range = ats.ETR_TTL)
board.setTriggerDelay(delay_samples=0) # no delay
board.setTriggerTimeOut(timeout_clocks = 0) # wait forever (0)
board.abortAsyncRead()

scanmode = 'unidirectional'
laser_frequency = 80000000
resonant_frequency = 8000
samples_per_pixel = int(np.round(laser_frequency / resonant_frequency*4 / 10000))

# configure the buffers
max_samples_per_record,bits_per_sample = board.getChannelInfo()
nlines = 512

if scanmode in [0,'uni','unidirectional']:
    post_trigger_samples = 5000*samples_per_pixel/4 # just one line...
    samples_per_record =  post_trigger_samples      # 10000/4 (1 sample every laser clock) samples per scan (back and forth)
    records_per_buffer = int(nlines)
else: # bidirectional
    post_trigger_samples = 9000*samples_per_pixel/4
    samples_per_record =  post_trigger_samples
    records_per_buffer = int(nlines/2)

# TODO: define the pixel_lookuptables here

# allocate the buffers
bytes_per_sample = 2;
nchannels = 4
nframes = 10 # continuous?

samples_per_buffer = samples_per_record * records_per_buffer * nchannels
bytes_per_buffer   = samples_per_buffer * bytes_per_sample

# allocate DMA buffer
nbuffers = 16
buffers = [ats.DMABuffer(c_sample_type = ctypes.c_uint16,
                         size_bytes = int(bytes_per_buffer)) for b in range(nbuffers)]

board.setRecordSize(preTriggerSamples = int(0),
                    postTriggerSamples = int(post_trigger_samples))


channel_selection = [1,1,1,1]

# configure acquisition
chan_sel = 0
for sel,channel in  zip(channel_selection, channel_list):
    if sel:
        chan_sel |=  channel        

board.beforeAsyncRead(
    channels = chan_sel,
    transferOffset = 0,
    samplesPerRecord = int(samples_per_record),
    recordsPerBuffer = int(records_per_buffer),
    recordsPerAcquisition = int(records_per_buffer*nframes),
    flags = ats.ADMA_EXTERNAL_STARTCAPTURE | ats.ADMA_NPT | ats.ADMA_INTERLEAVE_SAMPLES)

# post the buffers
for b in buffers:
    board.postAsyncBuffer(buffer=b.addr, bufferLength= int(bytes_per_buffer))

board.startCapture()
buffers_completed = 0
buffer_index = np.mod(buffers_completed, nbuffers)
# process a buffer and submit to the queue
b = buffers[buffers_completed % nbuffers]
try:
    board.waitAsyncBufferComplete(buffer=b.addr, timeout_ms = int(5000))
except Exception as err:
    if 'Timeout' in err.args:
        print('Timed out')
    else:
        raise(err)
print(b.buffer)

In [ ]:
samples_per_pixel

In [ ]:
nchannels = np.sum(channel_selection)

frame = np.array(b.buffer,dtype='uint16').reshape([nchannels,nlines,samples_per_pixel,-1]).sum(axis = 2)#.reshape([nchannels,-1,nlines]).astype('uint16')
frame.shape

In [ ]:
%matplotlib notebook
import pylab as plt
plt.imshow(frame[0])

In [ ]:
sbox_set_status_message(box.usb)

In [ ]:
box.scan()

In [ ]:
box.set_continuous_resonant(True)


In [ ]:
box.set_linescan_mode('bidi')


In [ ]:
from scanbox.nlw_controller import box_commands
box.cmd_queue.put(box_commands['version'])


In [ ]:
box.abort()